In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

# Predicting building demolition risk in Philadelphia's residential neighbourhoods, 2018-2021

#### Demolitions and property characteristics

In [2]:
#Get data on private demolitions post-2018 from API
demolitions = pd.read_csv("https://phl.carto.com/api/v2/sql?format=CSV&q=SELECT%20address,%20typeofwork%20FROM%20demolitions%20WHERE%20city_demo%20=%20%27NO%27%20AND%20start_date%20>=%20%272018-01-01%27")

demolitions.drop(index=demolitions[demolitions['typeofwork'] == 'TANKRI'].index, inplace=True) #drop tank removals

In [4]:
#Get selected features of all properties in Philadelphia
chunks = []
dtypes = {
    'lng': 'float64',
    'lat': 'float64',
    'location': 'object',
    'category_code_description': 'object',
    'interior_condition': 'object', #ordinal scale, therefore encoding this feature as a category
    'exterior_condition': 'object',
    'total_area': 'float64',
    'year_built': 'object',
    'parcel_number': 'object'
}

chunked_df = pd.read_csv('https://phl.carto.com/api/v2/sql?format=CSV&q=SELECT%20ST_X(the_geom)%20AS%20lng,%20ST_Y(the_geom)%20AS%20lat,%20location,%20category_code_description,%20interior_condition,%20exterior_condition,%20total_area,%20year_built,%20parcel_number%20FROM%20opa_properties_public', 
                         dtype=dtypes,
                         chunksize=40000)

for chunk in chunked_df:
    chunk['demolition'] = chunk['location'].isin(demolitions['address']).astype(np.int8) #create binary field encoding whether or not an address is associated with a demolition permit
    chunks.append(chunk)

properties = pd.concat(chunks)
properties.drop(index=properties[properties['category_code_description'] == 'Vacant Land'].index, inplace=True)

#Load in market value for properties in 2018
value18 = pd.read_csv('https://phl.carto.com/api/v2/sql?format=CSV&q=SELECT%20parcel_number,%20market_value%20FROM%20assessments%20WHERE%20year%20=%202018',
                     dtype={
                         'parcel_number': 'object',
                         'market_value': 'float64'
                     })

value18.loc[
    value18['market_value'] == 0
]

properties = pd.merge(properties, value18, how='left', on='parcel_number')

#Drop properties with text or null values in year_built field
properties.drop(index=properties[properties['year_built'].str.contains(r'[A-Za-z]+', na=True)].index, inplace=True)
properties['year_built'] = properties['year_built'].astype('int64')

#Drop properties built during or after 2018
properties.drop(index=properties[properties['year_built'] >= 2018].index, inplace=True)

del(value18) #save memory

#### Distance attributes

In [11]:
#Drop properties with null lat and long coordinates (they show up in pandas.info() as null, but not in the preview,
#but they still cause bugs in spatial operations)
properties.drop(index=properties['lat'][properties['lat'].isna()].index, inplace=True)

#Convert data frame to georeferenced dataframe to 
geo_properties = gpd.GeoDataFrame(properties,
                                  geometry=gpd.points_from_xy(properties.lng, properties.lat))

geo_properties.set_crs('epsg:4326', inplace=True) #set projection to WGS84
geo_properties.to_crs('epsg:2272', inplace=True) #reproject to NAD 1983 for southern PA
geo_properties.set_index('parcel_number', inplace=True) #set index

del(properties) #save memory

##### Distance to City Hall

In [15]:
#Get Easting and Northing coordinates for City Hall
city_hall = gpd.GeoSeries(Point(-75.1635112, 39.952335), crs=4326)
city_hall.to_crs('epsg:2272') #reproject to NAD 1983

0    POINT (2693536.305 236112.283)
dtype: geometry

In [16]:
#Turn City Hall into point object
city_hall = Point(2693536.305, 236112.283)

#Add field with distance to City Hall
geo_properties['dist_city_hall'] = geo_properties.distance(city_hall)

#Convert from feet to miles
geo_properties['dist_city_hall'] = geo_properties['dist_city_hall'] * 0.000189394

##### Distance to nearest public transportation stop

In [17]:
#Import public transportation shapefiles

##Trolley stops (within Philadelphia county)
trolley = gpd.read_file('https://services2.arcgis.com/9U43PSoL47wawX5S/arcgis/rest/services/Trolley_Stops1/FeatureServer/0/query?where=1%3D1&outFields=Route&outSR=4326&f=json')

##MFL stops
MFL = gpd.read_file('https://services2.arcgis.com/9U43PSoL47wawX5S/arcgis/rest/services/Market_Frankford_Line_Stations/FeatureServer/0/query?where=1%3D1&outFields=Route&outSR=4326&f=json')

##BSL stops
BSL = gpd.read_file('https://services2.arcgis.com/9U43PSoL47wawX5S/arcgis/rest/services/Broad_Street_Line_Stations/FeatureServer/0/query?where=1%3D1&outFields=Route&outSR=4326&f=json')

##Regional Rail stops (within Philadelphia County)
RR = gpd.read_file('https://services2.arcgis.com/9U43PSoL47wawX5S/arcgis/rest/services/Regional_Rail_Stations/FeatureServer/0/query?where=1%3D1&outFields=Line_Name,County&outSR=4326&f=json')

#Drop Regional Rail stations that aren't in Philadelphia
RR.drop(index=RR[RR['County'] != 'Philadelphia'].index, inplace=True)
RR.drop(columns='County', inplace=True)
RR.rename(columns={'Line_Name': 'Route'}, inplace=True)

##Concatenate transportation stops into one gdf of all transportation stops
transport = pd.concat([trolley, MFL, BSL, RR])
transport.to_crs('epsg:2272', inplace=True)

#delete individual transport files to save memory
del(trolley)
del(MFL)
del(BSL)
del(RR)

In [20]:
#Finding distance of each property to closest transport stop using scipy spatial cKDTree (fastest method)
def nearest_transport(props, transport):   
    '''Constructs cKDTree of transport stops, then returns Series of distances to closest transport stop
    for each property'''
    
    #Convert geometry columns to arrays
    prop_array = np.array(list(zip(props.geometry.x, props.geometry.y)))
    transport_array = np.array(list(zip(transport.geometry.x, transport.geometry.y)))
    
    #Construct cKDTree of transportation stops
    tree = cKDTree(transport_array)
    
    #Query tree to find distance to closest transport stop
    dist, idx = tree.query(prop_array,k=1) #dist is array of distances to closest point, idx is array of indices of closest points (not needed)
    
    #Since Easting and Northing are in feet, distances have to be converted to miles
    f = lambda x: x * 0.000189394
    dist_miles = f(dist) 
    
    return pd.Series(dist_miles, name='dist_to_transport', index=props.index)

geo_properties['dist_to_transport'] = nearest_transport(geo_properties, transport)

#### Neighbourhood attributes (change from 2000 to 2018)

In [23]:
#Function to extract Philadelphia from LTDB data (2000 Census)
def get_phl_data(fname):
    if fname == 'https://github.com/caranvr/DSSS-predicting-demolition/blob/main/data/LTDB_Std_2000_fullcount/LTDB_Std_2000_fullcount.csv?raw=true':
        cols = ['TRTID10', 'tract', 'county', 'POP00', 'NHWHT00', 'NHBLK00', 'HISP00', 'HU00', 'OWN00', 'RENT00']
    elif fname == 'https://github.com/caranvr/DSSS-predicting-demolition/blob/main/data/LTDB_Std_2000_Sample.csv?raw=true':
        cols = ['TRTID10', 'county', 'AG25UP00', 'COL00', 'HINC00']
    
    full_df = pd.read_csv(fname, usecols=cols)
    
    df = full_df.loc[
        full_df['county'] == 'Philadelphia County'
    ].copy()
    
    del(full_df)
    return df

#Load in full count and sample data for 2000, adjusted to 2010 boundaries
census00 = get_phl_data('https://github.com/caranvr/DSSS-predicting-demolition/blob/main/data/LTDB_Std_2000_fullcount/LTDB_Std_2000_fullcount.csv?raw=true')
census00_sample = get_phl_data('https://github.com/caranvr/DSSS-predicting-demolition/blob/main/data/LTDB_Std_2000_Sample.csv?raw=true')

phl00 = pd.merge(census00, census00_sample, on='TRTID10')

In [25]:
del(census00)
del(census00_sample)

In [27]:
phl00.set_index('TRTID10', inplace=True)

In [28]:
#Get Census data from API (ACS 5-year estimates from 2013 to 2018)
key = 'c0c4631460ee7868c6c89b3e7d58e9941bd285d7' #API access key
variables = ['B01001_001E', 'B03002_003E', 'B03002_004E', 'B03001_003E', 'B25003_001E', 'B25003_002E', 'B25003_003E', 'B15002_001E',
            'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E', 'B19013_001E']
state = '42' #state code
county = '101' #county code

url = 'https://api.census.gov/data/2018/acs/acs5?get=' + ",".join(variables) + '&for=tract:*&in=state:' + state + '&in=county:' + county + '&key=' + key
phl18 = pd.read_json(url)

#Column names are in first row - make sure column names are in the right place
phl18.columns = phl18.iloc[0]
phl18.drop(index=phl18.index[0], axis=0, inplace=True)

#Convert numeric variables to ints
for v in variables:
    if v != 'B19013_001E': #all variables except for median income
        phl18[v] = phl18[v].astype('float').astype('Int64')
    else:
        phl18[v] = phl18[v].astype('float')
        
#Create new index for joining 2000 and 2013-18 Census data
phl18['TRTID10'] = (phl18['state'] + phl18['county'] + phl18['tract']).astype('int')
phl18.set_index('TRTID10', inplace=True)

#Drop non-residential Census tracts
phl18.drop(index=phl18[phl18['B01001_001E'] == 0].index, inplace=True)



In [32]:
#Check for nulls, since they aren't encoded as nans
check_nulls = phl18[variables].lt(0).sum(axis=1)
null_rows = check_nulls[check_nulls > 0

In [33]:
#Convert nulls to actual nans so they don't mess up the calculations
phl18['B19013_001E'] = phl18['B19013_001E'].replace(-666666666, np.nan)

In [34]:
#Add up post-college education fields
phl18['COL18'] = phl18['B15003_022E'] + phl18['B15003_023E'] + phl18['B15003_024E'] + phl18['B15003_025E']

#Drop constituent post-college education fields
phl18.drop(columns=['B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E'], inplace=True)

#Join 2000 and 2018 Census variables
phl_change = pd.merge(phl18, phl00, how='left', left_index=True, right_index=True)

#Change household income from 2000 to 2018 dollars
phl_change['HINC00'] = phl_change['HINC00'].astype('float') * 1.5382

In [35]:
#Join Census tract data from 2018 to each property (load Census tract shapefile, then gpd sjoin)
phl_tracts = gpd.read_file('https://opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.geojson')

phl_tracts.rename(columns={'GEOID10': 'TRTID10'}, inplace=True)
phl_tracts['TRTID10'] = phl_tracts['TRTID10'].astype('int')
phl_tracts.set_index('TRTID10', inplace=True)

#Reproject to allow for a spatial join between geo_properties and tracts
phl_tracts.to_crs('epsg:2272', inplace=True)

phl_change_geo = pd.merge(phl_tracts, phl_change, how='inner', left_index=True, right_index=True)

In [36]:
#Spatially join properties to Census tracts (only including residential Census tracts)
geo_properties = gpd.sjoin(geo_properties, phl_change_geo, op='within')

drop_cols = ['OBJECTID', 'STATEFP10', 'COUNTYFP10', 'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
            'INTPTLAT10', 'INTPTLON10', 'LOGRECNO', 'state', 'county', 'tract_x', 'county_x', 'tract_y', 'county_y']

geo_properties.drop(columns=drop_cols, inplace=True)

In [45]:
geo_properties.to_csv('geo_props_final.csv', chunksize=50000)